# 非特征点法VIO

## 一、介绍
非特征点法的视觉里程计有光流法(Optical Flow)和直接法(Direct Method)两种。
* 光流法需提取具有区别性的特征点，再用特征点进行光流估计得到对应点对(而不是计算描述子和匹配特征点)。
* 直接法无需提取特征点，直接法并不需要知道点与点之间的对应关系，而是通过最小化光度误差(Photometric error)来求得它们。与特征点法只能重构稀疏特征地图相比，直接法可以恢复稠密或半稠密结构的能力。

## 二、光流法
### 2. 1 介绍

光流是描述像素随时间在图像中运动的模型，计算部分像素的运动称为**稀疏光流**，计算所有像素称为**稠密光流**。稀疏光流以Lucas-Kanade光流为代表，稠密光流以Horn-Schunck光流为代表，这里主要介绍LK光流。

### 2.2 LK光流
光流法可以看成流体力学中欧拉法在图像中的应用，欧拉法着眼于空间点，描述各个时刻，各个空间点中流体质点物理量的变化。对应LK光流即定义了在x, y, t上的“像素场”

$$I(x, y, t)$$

我们希望通过这个像素场估计特征点的位置变化，需要引入光流法的基本假设——**灰度不变假设**：同一个点的像素灰度值在各个图像中不变，则对于$t$时刻位于$(x, y)$的像素，设其在$t + dt$时刻移动到$(x + dx, y + dy)$：
$$I(x + dx, y + dy, t + dt) \approx I(x, y, t) + \frac{\partial I}{\partial x}dx+ \frac{\partial I}{\partial y}dy+ \frac{\partial I}{\partial t}dt$$
则由
$$I(x + dx, y + dy, t + dt) = I(x, y, t)$$
有

$$\frac{\partial I}{\partial t} = -\frac{\partial I}{\partial x}\frac{dx}{dt}- \frac{\partial I}{\partial y}\frac{dy}{dt}$$

写成矩阵形式

$$[I_x, I_y]\left[ \begin{matrix}u \\ v\end{matrix}\right] = -I_t$$
$$u = \frac{dx}{dt}, \ v = \frac{dy}{dt}$$

其中$I_x, I_y, I_t$已知，需要求解像素运动$u, v$。在LK光流中，**假设在某一个窗口中的所有像素具有相同的运动**，这样就可以把u，v的求解转化成一个最小二乘问题求解。
除了最小二乘外，LK光流还可以用高斯牛顿法求解如下优化问题

$$\min_{\Delta x, \Delta y}||I_1(x, y) - I_2(x + \Delta x, y + \Delta y)||_2^2$$

如果相机运动较快，两张图片差异明显，则单层图像光流法容易达到局部极小值，可以引入图像尺度金字塔改善。对原始图像进行不同比率的降采样，然后从顶层图像开始计算，把上层的追踪结果作为下一层光流的初始值。由于上层的图像相对粗糙，所以这个过程也称为**由粗到精**(coarse-to-fine)的光流。
### 2.3 光流法小结
LK光流能够直接得到特征点的对应关系，随后再通过ICP，对极几何，PnP等算法计算相机位姿。然而，光流对图像的连续性和光照稳定性要求更高，要求相机运动较平滑(或采样频率较高)。

## 三、直接法(Direct Method)
### 3.1 介绍
直接法的假设也是**灰度不变假设**，即一个空间点在不同视角下成像的灰度不变。直接法直接对相机位姿进行优化：

$$\min_{R, t}\sum_{i}||I_1(\frac{1}{z_1^i}KP_i)- I_2(\frac{1}{z_2^i}K(RP_i + T))||_2^2$$

那如何得到三维点$P_i$呢，在RGBD相机和双目相机下可以直接得到像素对应的三维坐标，而对于单目相机则还需考虑深度带来的不确定性，需要进行深度估计。

### 3.1 分类
根据三维点的来源，对直接法进行分类：
1. P来自稀疏关键点，称为稀疏直接法，稀疏直接法不必计算描述子，只使用数百像素，因此速度最快，但只能计算稀疏的重构。
2. P来自部分像素，即舍弃像素梯度不明显的地方(对运动估计没有贡献)，只考虑带有梯度的像素点。称为半稠密(Semi-Dense)直接法，可以重构一个半稠密的结构。
3. P来自所有像素，称为稠密直接法，需要GPU加速。

## 三、参考资料
[1] 视觉SLAM十四讲——从理论到实践 第七、八章